In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 1.x
#import the necessary libraries
"""
We will be using Tensorflow of version 1.x
"""
import pandas as pd
import numpy as np
import re
import datetime
from zipfile import ZipFile
from os.path import expanduser, exists
import time
import json
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras import models
from keras import layers
from keras import optimizers
from keras import activations
from keras import losses
from keras import metrics
from keras.models import load_model
from keras import layers
from keras import activations
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Input
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.layers.embeddings import Embedding
from keras.utils.data_utils import get_file
from keras.models import Model
from keras.layers import LSTM
from keras.layers import merge
from keras.layers import Dropout
from keras.layers import concatenate
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Lambda,TimeDistributed,Dot,dot






In [0]:
#Read the data:
"""
i/p: Path of the folder where the dataset is placed
o/p: dataframe
"""
path = "/content/drive/My Drive/Project/Data/" #give the path where questions.csv is saved
df = pd.read_csv(path + "questions.csv")

In [0]:
#create an extra copy of the dataframe
train_df = df.copy()
train_df

In [0]:
train_df_copy = train_df.copy()
train_df_copy.describe()


In [0]:
#final dataframe copy for training
train_DF = train_df.copy()

In [0]:
#Store columns as lists to make it model ready
"""
i/p: Columns of DataFrame
o/p: list of values for each column
"""
q1_l = train_DF['question1'].tolist()
q1_l = [str(q) for q in q1_l]
q2_l = train_DF['question2'].tolist()
q2_l = [str(q) for q in q2_l]
label_list = train_DF['is_duplicate'].tolist()


In [0]:
#Tokenize on the entire data,get the word sequences,word_indx.This will be the input to the model
"""
i/p: q1_list+q2_list
o/p:dict_word_indx , key = word and value = word_index
""" 
total_ques = q1_l + q2_l
token = Tokenizer(num_words=100000)
token.fit_on_texts(total_ques)

q1_seq = token.texts_to_sequences(q1_l)
q2_seq = token.texts_to_sequences(q2_l)
word_indx = token.word_index
dict_word_indx = word_indx
with open(path + 'word_indx.json', 'w') as dfile:
    json.dump(dict_word_indx, dfile)

In [0]:
#download GLOVE embeddings:
# Citations: Stackoverflow.com,'http://nlp.stanford.edu/data/glove.840B.300d.zip'
"""
i/p: GloVe
o/p: embeddings_index dict
"""


GLOVE_URL = 'http://nlp.stanford.edu/data/glove.840B.300d.zip'

if not exists(expanduser('~/.keras/datasets/glove.840B.300d.zip')):
    zipfile = ZipFile(get_file('glove.840B.300d.zip', GLOVE_URL))
    zipfile.extract('glove.840B.300d.txt', path=expanduser('~/.keras/datasets/'))
    
print("Processing", 'glove.840B.300d.txt')

embed_indx = {}

with open(expanduser('~/.keras/datasets/glove.840B.300d.txt'), encoding='utf-8') as filee:
    for l in filee:
        val = l.split(' ')
        w = val[0]
        embed = np.asarray(val[1:], dtype='float32')
        embed_indx[w] = embed

print('Word embeddings: %d' % len(embed_indx))

Processing glove.840B.300d.txt
Word embeddings: 2196016


In [0]:
#Embedding matrix
"""
I/p: word sequences
o/p: embedded vectors of words
"""
dim_embed = 300
word_max = 100000
nwords = min(word_max, len(word_indx))
w_embed_matr = np.zeros((nwords + 1, dim_embed))
for w, i in word_indx.items():
    if i > word_max:
        continue
    embedd_vector = embed_indx.get(w)
    if embedd_vector is not None:
        w_embed_matr[i] = embedd_vector

#print('Null word embeddings: %d' % np.sum(np.sum(w_embed_matr, axis=1) == 0))

Null word embeddings: 29273


In [0]:

#q1_seq[0:5]

[[2, 3, 1, 1228, 57, 1228, 2582, 7, 575, 8, 763, 383, 8, 35],
 [2, 3, 1, 558, 10, 14302, 13600, 5, 21317, 4565],
 [4, 13, 5, 217, 1, 440, 10, 17, 361, 1827, 200, 146, 6, 2773],
 [16, 72, 5, 2774, 307, 2757, 4, 13, 5, 648, 19],
 [23, 49, 7130, 8, 230, 35502, 1893, 2046, 10573, 12, 1928, 10923, 6461]]

In [0]:
#Padding the sequences with zeros, length of each sequence = max_length of sequence
"""
i/p:sequence
o/p:padded sequence,X- features,y-label
"""
max_len_seq = 0
for q in q1_seq:
    if(len(q) > max_len_seq):
        max_len_seq = len(q)
max_len_seq
max_len_seq= 130
q1_d = pad_sequences(q1_seq, maxlen=max_len_seq)
q2_d = pad_sequences(q2_word_seq, maxlen=max_len_seq)
label = np.array(label_list, dtype=int)
X = np.stack((q1_d, q2_d), axis=1)
y = label


127

In [0]:
#Model Preparation
#Split the data as train,valid and test in the ratio 7:1:2
train_ratio = 0.70
validation_ratio = 0.10
test_ratio = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-train_ratio)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]
Q1_val = X_val[:,0]
Q2_val = X_val[:,1]


In [0]:
#construct the siamese neural network using the manhattan distance:
def exp_manhattan_distance(l1, l2):
    '''Manhattan distance as similarity measure of inputs'''
    return K.exp(-K.sum(K.abs(l1-l2), axis=1, keepdims=True))



In [113]:
#Building the model:
"""
i/p: input sequence of question1 and question 2
o/p: 2LSTMs combined by manhattan distance as the output,a single value :0 or 1
"""
q1 = Input(shape=(max_len_seq,))
q2 = Input(shape=(max_len_seq,))
ques1 = Embedding(nwords + 1, 
                 dim_embed, 
                 weights=[w_embed_matr], 
                 input_length=max_len_seq, 
                 trainable=False)(q1)


ques2 = Embedding(nwords + 1, 
                 dim_embed, 
                 weights=[w_embed_matr], 
                 input_length=max_len_seq, 
                 trainable=False)(q2)


n_hidden = 100
s_lstm = LSTM(n_hidden)
left = s_lstm(ques1)
right = s_lstm(ques2)
distance = Lambda(function=lambda x: exp_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left, right])

#final model
model = Model(input = [q1,q2], output = [distance])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


In [0]:
#save the model
model_j = model.to_json()
with open(path + 'model.json', 'w') as json_f:
    json_f.write(model_j)

In [0]:
#Train the model with inputs as Q1_train,Q2_train
print("Start time", datetime.datetime.now())
st_time = time.time()
his_ = model.fit([Q1_train, Q2_train],
                    y_train,
                    epochs=25,
                    validation_data=([Q1_val, Q2_val], y_val),
                    verbose=1,
                    batch_size=64)
end_time = time.time()
print("End Time", datetime.datetime.now())
print("Total Time: %f" % ((end_time - st_time) / 60.))

In [0]:
#fit the model with validation,test to get the necessary evaluation metrics
values = model.evaluate(x=[Q1_val, Q2_val], y=y_val)
print("Loss val: " + str(values[0]))
print("Accuracy val: " + str(values[1]))

values_t = model.evaluate(x=[Q1_test, Q2_test], y=y_test)
print("Loss test: " + str(values_t[0]))
print("Accuracy test: " + str(values_t[1]))



In [0]:
#Model evaluation
"""
i/p:validation, test
o/p: results_val.txt,results_test.txt
"""

yhat_probs_val = model.predict([Q1_val, Q2_val], verbose=0)
yhat_probs_test = model.predict([Q1_test, Q2_test], verbose=0)
y_pred = np.ravel(yhat_probs_val).tolist()
df_pred = pd.DataFrame()
df_pred['pred_classes']= y_pred
y_val = np.ravel(y_val).tolist()
df_pred['True Y_val'] = y_val
excel_ = df_pred.to_excel(path + "Pred_val_2.xlsx",index = None, header = True)

y_pred_t = np.ravel(yhat_probs_test).tolist()
df_pred_t = pd.DataFrame()
df_pred_t['pred_classes']= y_pred_t
y_test = np.ravel(y_test).tolist()
df_pred_t['True Y_test'] = y_test
excel_t = df_pred_t.to_excel(path + "Pred_test.xlsx",index = None, header = True)


y_pr_val = (df_pred['pred_classes']).tolist()
y_val_tr = (df_pred['True Y_val']).tolist()
y_pr_class_val = [(lambda i: 1 if i >= 0.5 else 0)(i) for i in y_pr_val]

y_pr_test = (df_pred_t['pred_classes']).tolist()
y_test_tr = (df_pred_t['True Y_test']).tolist()
y_pr_class_test = [(lambda i: 1 if i >= 0.5 else 0)(i) for i in y_pr_test]

In [0]:

with open(path + "Result_SiMaLSTM_val.txt","a",encoding = "UTF-8") as f:
  accuracy = accuracy_score(y_val_tr, y_pr_class_val)
  print('Accuracy: %f' % accuracy)
  f.write("Accuracy:" + str(accuracy) + "\n")
  precision = precision_score(y_val_tr, y_pr_class_val)
  print('Precision: %f' % precision)
  f.write("precision:" + str(precision) + "\n")
  # recall: tp / (tp + fn)
  recall = recall_score(y_val_tr, y_pr_class_val)
  print('Recall: %f' % recall)
  f.write("recall:" + str(recall) + "\n")
  # f1: 2 tp / (2 tp + fp + fn)
  f1 = f1_score(y_val_tr, y_pr_class_val)
  print('F1 score: %f' % f1)
  f.write("f1 score:" + str(f1) + "\n")
  auc = roc_auc_score(y_val_tr, y_pr_class_val)
  print('ROC score: %f' % auc)
  f.write("ROC score:" + str(auc) + "\n")
  # confusion matrix
  matrix = confusion_matrix(y_val_tr, y_pr_class_val)
  print(matrix)



with open(path + "Result_SiMaLSTM_test.txt","a",encoding = "UTF-8") as f:
  accuracy = accuracy_score(y_test_tr, y_pr_class_test)
  print('Accuracy: %f' % accuracy)
  f.write("Accuracy:" + str(accuracy) + "\n")
  precision = precision_score(y_val_tr, y_pr_class_val)
  print('Precision: %f' % precision)
  f.write("precision:" + str(precision) + "\n")
  # recall: tp / (tp + fn)
  recall = recall_score(y_val_tr, y_pr_class_val)
  print('Recall: %f' % recall)
  f.write("recall:" + str(recall) + "\n")
  # f1: 2 tp / (2 tp + fp + fn)
  f1 = f1_score(y_val_tr, y_pr_class_val)
  print('F1 score: %f' % f1)
  f.write("f1 score:" + str(f1) + "\n")
  auc = roc_auc_score(y_val_tr, y_pr_class_val)
  print('ROC score: %f' % auc)
  f.write("ROC score:" + str(auc) + "\n")
  # confusion matrix
  matrix = confusion_matrix(y_val_tr, y_pr_class_val)
  print(matrix)
